In [70]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numbers import Number
import math, cmath

In [115]:
def zin_lossless(z0: Number, zl: Number, beta: Number, l: Number) -> Number:
    tg = cmath.tan(beta * l)
    num = z0 * (zl + 1j*z0*tg)
    den = z0 + 1j*zl*tg
    return num/den

def zin_lossless(z0: Number, zl: Number, beta_times_l: Number) -> Number:
    tg = cmath.tan(beta_times_l)
    num = z0 * (zl + 1j*z0*tg)
    den = z0 + 1j*zl*tg
    return num/den

def V_lossless(V0: Number, reflection_coefficient: Number, beta: Number, z: Number):
    V = V0*(cmath.exp(-1j*beta*z) + reflection_coefficient*cmath.exp(1j*beta*z))
    return V

def I_lossless(V0: Number, Z0: Number, reflection_coefficient: Number, beta: Number, z: Number):
    I = (V0/Z0)*(cmath.exp(-1j*beta*z) - reflection_coefficient*cmath.exp(1j*beta*z))
    return I

def reflection_coefficient_(z0: Number, zl: Number) -> Number:
    num = zl - z0
    den = zl + z0
    return num/den

def swr_(reflection_coefficient):
    return (1 + abs(reflection_coefficient))/(1 - abs(reflection_coefficient))

def propagation_constant_(R, L, G, C, w):
    term_1 = R + 1j*w*L
    term_2 = G + 1j*w*C
    return cmath.sqrt(term_1*term_2)

def impedance_(R, L, G, C, w):
    a = R + 1j*w*L
    b = G + 1j*w*C
    return cmath.sqrt(a/b)

def atenuation(alpha, l, in_db = False):
    value = math.exp(-1 * alpha * l)
    if in_db:
        return -2 * math.log10(value)

    return value

def unit_to_db(value):
    return 20 * math.log10(value)

def display_complex(z):
    msg = (f'z = {z.real}+{z.imag}j in rectangular coordinates \n'
           f'or z = {abs(z)}*e^{cmath.phase(z)}j in polar coordinates')
    print(msg)
    

In [103]:
R1 = 4
L1 = 0.5 * 1e-6
G1 = 0.02
C1 = 200 * 1e-12
w1 = 800 * 1e6

gama1 = propagation_constant_(R1, L1, G1, C1, w1)
alpha1 = gama1.real

l1 = 0.3
atenuation1 = atenuation(alpha1, l1)
atenuation1_db = atenuation(alpha1, l1, True)
print("gama: " + str(gama1))
print(f'atenuation after {l1} m is {atenuation1_db} dB')

gama: (0.5391135357553036+8.01315439789067j)
atenuation after 0.3 m is 0.14048042021872792 dB


In [96]:
# Lista 1

# 2.2 (Pozar 3 ed.)
L = 0.2*1e-6
C = 300*1e-12
R = 5
G = 0.01 
w = 500*1e6

pc_lossy = propagation_constant_(R, L, G, C, w)
ch_impedance_lossy = impedance_(R, L, G, C, w)
print(pc_lossy)
display_complex(pc_lossy)
print(ch_impedance_lossy)

pc_lossless = propagation_constant_(0, L, 0, C, w)
ch_impedance_lossless = impedance_(0, L, 0, C, w)
print(pc_lossless)
print(ch_impedance_lossless)

(0.22591621095124334+3.873117366459551j)
z = 0.22591621095124334+3.873117366459551j in rectangular coordinates 
or z = 3.879700538539171*e^1.5125330470460134j in polar coordinates
(25.806494118515268+0.21432486822606223j)
3.872983346207417j
(25.819888974716115+0j)


In [97]:
# 2.6
def L_coax(permeability, b, a):
    return (permeability*math.log(b/a))/ (2*math.pi)

def C_coax(permittivity_real, b, a):
    return (2*math.pi*permittivity_real)/math.log(b/a)

def Rs(w, condutivity, permeability):
    return math.sqrt((w*permeability)/(2*condutivity))

def R_coax(Rs, b, a):
    return (Rs/(2*math.pi))*((1/a) + (1/b))

def G_coax(w, permittivity_imag, b, a):
    return (2*math.pi*w*permittivity_imag)/math.log(b/a)

# Vacuum constants
permeability = 4*math.pi*1e-7
permittivity = 8.854*1e-12

# Frequency
f = 1*1e9 # 1GHz
w = 2*math.pi*f

# Dimensions
b = 3.02*1e-3 # outer radius
a = 0.91*1e-3 # inner radius

# Conductor is Copper:
condutivity_copper = 5.813*1e7
permeability_copper = permeability
Rs_copper = Rs(w, condutivity_copper, permeability_copper)
print(f'Rs={Rs_copper}')

# Dieletric is Teflon
permittivity_relative_teflon = 2.08
loss_tan_teflon = 4*1e-4
permittivity_real_teflon = permittivity_relative_teflon * permittivity
permittivity_imag_teflon = loss_tan_teflon * permittivity_real_teflon

L = L_coax(permeability_copper, b, a)
C = C_coax(permittivity_real_teflon, b, a)
R = R_coax(Rs_copper, b, a)
G = G_coax(w, permittivity_imag_teflon, b, a)
print(f'L={L} C={C} R={R} G={G}')

impedance = impedance_(R, L, G, C, w)
propagation_constant = propagation_constant_(R, L, G, C, w)
print(f'Z={impedance} propagation_constant={propagation_constant}')

Rs=0.008240996067111548
L=2.39913502171604e-07 C=9.646239181115444e-11 R=1.8756165388694308 G=0.0002424364331692984
Z=(49.87104220166847-0.02105193129526322j) propagation_constant=(0.024849945426179916+30.22638887076501j)


In [124]:
# 2.8

z0 = 75
zl = 30 - 1j*20

reflection_coefficient = reflection_coefficient_(z0, zl)
print(cmath.polar(reflection_coefficient))

swr = swr_(reflection_coefficient)
print(swr)

reflection_coefficient_at_input = reflection_coefficient * cmath.exp(1.2*cmath.pi*1j)
print(cmath.polar(reflection_coefficient_at_input))

beta_times_l = 0.6*cmath.pi
zin = zin_lossless(z0, zl, beta_times_l)
print(zin)

(0.4607101359007234, -2.5351468187057935)
2.7085807339257255
(0.4607101359007234, 1.2347643656019582)
(202.9897237775293-5.192938734670089j)
